# Nim Policy Search

Nim is a simple game where two players take turns removing objects from a pile. The player who removes the last object loses. The game is described in detail [here](https://en.wikipedia.org/wiki/Nim). There is a mathematical strategy to win Nim, by ensuring you always leave the opponent with a nim-sum number of objects (groups of 1, 2 and 4).

In this notebook, we will play nim-sum using the following agents:
1. An agent using fixed rules based on nim-sum
2. An agent using evolved rules
3. An agent using minmax
4. An agent using reinforcement learning

> Sidharrth Nagappan, 2022

In [1]:
import logging
import random

In [2]:
class Nim:
    def __init__(self, num_rows: int, k: int = None):
        self.num_rows = num_rows
        self._k = k
        self.rows = [i*2+1 for i in range(num_rows)]

    def nimming_remove(self, row: int, num_objects: int):
        assert self.rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self.rows[row] -= num_objects
    
    def goal(self) -> bool:
        return sum(self.rows) == 0

In [34]:
class RandomPlayer:
    def __init__(self):
        self.num_moves = 0
    
    def play(self, nim: Nim):
        # Chose a random row and a random number of pieces
        row = random.randint(0, len(nim.rows)-1)
        if nim.rows[row] == 0:
            while nim.rows[row] == 0:
                row = random.randint(0, len(nim.rows)-1)
        to_remove = random.randint(1, nim.rows[row])
        self.num_moves += 1
        nim.nimming_remove(row, to_remove)        

In [35]:
from copy import deepcopy
from itertools import accumulate
from operator import xor

# 3.1: Agent Using Fixed Rules
class ExpertFixedRuleAgent:
    '''
    Play the game of Nim using a fixed rule 
    (always leave nim-sum at the end of turn)
    '''
    def __init__(self):
        self.num_moves = 0
    
    def nim_sum(self, nim: Nim):
        '''
        Returns the nim sum of the current game board
        by taking an XOR of all the rows.
        Ideally, agent should try to leave nim sum of 0 at the end of turn
        '''
        *_, result = accumulate(nim.rows, xor)
        return result
        # return sum([i^r for i, r in enumerate(nim._rows)])
    
    def play(self, nim: Nim):
        # remove objects from row to make nim-sum 0
        nim_sum = self.nim_sum(nim)
        all_possible_moves = [(r, o) for r, c in enumerate(nim.rows) for o in range(1, c+1)]     
        move_found = False
        for move in all_possible_moves:
            replicated_nim = deepcopy(nim)
            replicated_nim.nimming_remove(*move)
            if self.nim_sum(replicated_nim) == 0:
                nim.nimming_remove(*move)
                move_found = True
                break
        # if a valid move not found, return random move
        if not move_found:
            move = random.choice(all_possible_moves)
            nim.nimming_remove(*move)
        
        # print(f"Move {self.num_moves}: Removed {move[1]} objects from row {move[0]}")
        self.num_moves += 1
        
        # for i, row in enumerate(nim._rows):
        #     if row ^ nim_sum < row:
        #         nim.nimming_remove(i, row - (row ^ nim_sum))
        #         return        
    
    # remove objects to leave nim-sum at the end of turn
    # loop over rows and split objects into 1, 2, 4
    # remove left over objects

In [36]:
# random agent vs fixed rule agent

nim = Nim(20)
p1 = RandomPlayer()
p2 = ExpertFixedRuleAgent()
current_player = 1

while not nim.goal():
    if current_player == 1:
        p1.play(nim)
        print(f"Random plays: {nim.rows}")
        current_player = 2
    else:
        p2.play(nim)
        print(f"Expert plays: {nim.rows}")
        current_player = 1

Random plays: [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 21, 33, 35, 37, 39]
Expert plays: [1, 3, 5, 7, 3, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 21, 33, 35, 37, 39]
Random plays: [1, 3, 5, 7, 3, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 21, 33, 33, 37, 39]
Expert plays: [1, 1, 5, 7, 3, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 21, 33, 33, 37, 39]
Random plays: [1, 1, 5, 7, 3, 8, 13, 15, 17, 19, 21, 23, 25, 27, 29, 21, 33, 33, 37, 39]
Expert plays: [1, 1, 5, 4, 3, 8, 13, 15, 17, 19, 21, 23, 25, 27, 29, 21, 33, 33, 37, 39]
Random plays: [1, 1, 5, 4, 3, 8, 13, 15, 17, 19, 21, 23, 5, 27, 29, 21, 33, 33, 37, 39]
Expert plays: [1, 1, 5, 4, 3, 8, 13, 15, 13, 19, 21, 23, 5, 27, 29, 21, 33, 33, 37, 39]
Random plays: [0, 1, 5, 4, 3, 8, 13, 15, 13, 19, 21, 23, 5, 27, 29, 21, 33, 33, 37, 39]
Expert plays: [0, 0, 5, 4, 3, 8, 13, 15, 13, 19, 21, 23, 5, 27, 29, 21, 33, 33, 37, 39]
Random plays: [0, 0, 5, 4, 3, 5, 13, 15, 13, 19, 21, 23, 5, 27, 29, 21, 33, 33, 37, 39]
Expert plays: [0, 0, 5

In [ ]:
# 3.2: Agent Using Evolved Rules (Randomly Chooses Between Strategies)
import numpy as np

class EvolvedAgent1:
    '''
    Plays Nim using a set of rules that are evolved
    '''
    def __init__(self):
        self.num_moves = 0
    
    def nim_sum(self, nim: Nim):
        '''
        Returns the nim sum of the current game board
        by taking an XOR of all the rows.
        Ideally, agent should try to leave nim sum of 0 at the end of turn
        '''
        *_, result = accumulate(nim.rows, xor)
        return result

    def play_nim(self, nim: Nim, prob_list: list):
        '''
        GA can choose between the following strategies:
        1. Randomly pick any row and any number of elements from that row
        2. Pick the shortest row
        3. Pick the longest row
        4. Pick based on the nim-sum of the current game board
        '''
        all_possible_moves = [(r, o) for r, c in enumerate(nim.rows) for o in range(1, c+1)]
        strategies = {
            'nim_sum': random.choice([move for move in all_possible_moves if self.nim_sum(deepcopy(nim).nimming_remove(*move)) == 0]),
            'random': random.choice(all_possible_moves),
            'all_elements_shortest_row': (nim.rows.index(min(nim.rows)), min(nim.rows)),
            '1_element_shortest_row': (nim.rows.index(min(nim.rows)), 1),
            'random_element_shortest_row': (nim.rows.index(min(nim.rows)), random.randint(1, min(nim.rows))),
            'all_elements_longest_row': (nim.rows.index(max(nim.rows)), max(nim.rows)),
            '1_element_longest_row': (nim.rows.index(max(nim.rows)), 1),
            'random_element_longest_row': (nim.rows.index(max(nim.rows)), random.randint(1, max(nim.rows))),
        }
        
        p = random.random()
        strategy = None
        if p < prob_list[0]:
            strategy = strategies['random']
        elif p >= prob_list[0] and p < prob_list[1]:
            strategy = random.choice([strategies['all_elements_shortest_row'], strategies['1_element_shortest_row'], strategies['random_element_shortest_row']])
        elif p >= prob_list[1] and p < prob_list[2]:
            strategy = random.choice([strategies['all_elements_longest_row'], strategies['1_element_longest_row'], strategies['random_element_longest_row']])
        else:
            strategy = strategies['nim_sum']
        
        nim.nimming_remove(*strategy)
        self.num_moves += 1
        return sum(nim.rows)

    def play(self, nim: Nim):
        '''
        Play the game of Nim using the evolved rules
        '''
        prob_list = [0.25, 0.5, 0.75, 1]
        prob_list = self.evolve_probabilities(nim, prob_list, 20, 5)
        self.play_nim(nim, prob_list)

    def crossover(self, p1, p2):
        '''
        Crossover between two parents
        '''
        return np.random.choice(p1 + p2, size=4, replace=True)    

    def evolve_probabilities(self, nim: Nim, prob_list: list, num_generations: int, num_children: int):
        '''
        Evolve the probabilities of the strategies
        '''
        # create initial population
        population = [prob_list for _ in range(num_children)]
        # create initial fitness scores
        fitness_scores = [self.play(nim, p) for p in population]
        # create initial parents
        parents = [population[i] for i in np.argsort(fitness_scores)[:2]]
        # create new population
        new_population = []
        for _ in range(num_generations):
            # create children
            for _ in range(num_children):
                p1 = random.choice(parents)
                p2 = random.choice(parents)
                child = self.crossover(p1, p2)
                # child = []
                # for i in range(len(parents[0])):
                #     # crossover between parents
                    
                #     child.append(random.choice(parents)[i])
                new_population.append(child)
            # create fitness scores
            fitness_scores = [self.play_nim(nim, p) for p in new_population]
            # create new parents
            parents = [new_population[i] for i in np.argsort(fitness_scores)[:2]]
            # create new population
            new_population = []
        return parents[0]

In [ ]:
import math

class MinMaxAgent:
    def __init__(self):
        self.num_moves = 0
    
    def nim_sum(self, nim: Nim):
        '''
        Returns the nim sum of the current game board
        by taking an XOR of all the rows.
        Ideally, agent should try to leave nim sum of 0 at the end of turn
        '''
        *_, result = accumulate(nim.rows, xor)
        return result

    def minmax(self, nim: Nim, depth: int, maximizing_player: bool):
        '''
        Minimax algorithm to find the best move
        '''
        if depth == 0 or nim.goal():
            return self.nim_sum(nim)
        
        if maximizing_player:
            value = -math.inf
            for r, c in enumerate(nim.rows):
                for o in range(1, c+1):
                    # make copy of nim object before running a nimming operation
                    replicated_nim = deepcopy(nim)
                    replicated_nim.nimming_remove(r, o)
                    value = max(value, self.minmax(replicated_nim, depth-1, False))
            return value
        else:
            value = math.inf
            for r, c in enumerate(nim.rows):
                for o in range(1, c+1):
                    # make copy of nim object before running a nimming operation
                    replicated_nim = deepcopy(nim)
                    replicated_nim.nimming_remove(r, o)
                    value = min(value, self.minmax(replicated_nim, depth-1, True))
            return value

    def play(self, nim: Nim):
        '''
        Minimax algorithm to find the best move
        '''
        best_move = None
        best_value = -math.inf
        for r, c in enumerate(nim.rows):
            for o in range(1, c+1):
                replicated_nim = deepcopy(nim)
                replicated_nim.nimming_remove(r, o)
                value = self.minmax(replicated_nim, 5, False)
                if value > best_value:
                    best_value = value
                    best_move = (r, o)
        nim.nimming_remove(*best_move)
        self.num_moves += 1

In [ ]:
from typing import Callable


NUM_MATCHES = 10
NIM_SIZE = 10

def evaluate(strategy: Callable) -> float:
    opponent = (strategy, optimal_startegy)
    won = 0

    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = opponent[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won / NUM_MATCHES